## Week 3 Homework 
---
> **Important Note:**

You can load the data however you would like, but keep the files in .GZ Format. 
If you are using orchestration such as Airflow or Prefect do not load the data into Big Query using the orchestrator.
Stop with loading the files into a bucket. 

> **NOTE:** 

You can use the CSV option for the GZ files when creating an External Table

> **SETUP:**

Create an external table using the fhv 2019 data. 
Create a table in BQ using the fhv 2019 data (do not partition or cluster this table). 
Data can be found here: https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv 

In [2]:
#imports
import pandas as pd
from google.cloud import bigquery
from time import time
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering/week_3_data_warehouse/GOOGLE_APPLICATION_CREDENTIALS.json"
print("Setup Complete") 

Setup Complete


In [3]:
#create a client
client = bigquery.Client()
# Construct a reference to the "ny_taxi_bucket_de_2023" dataset
dataset_ref = client.dataset(dataset_id="ny_taxi", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference to "ny_taxi_tripdata_2019"
table_ref = dataset_ref.table("ny_taxi_tripdata_2019")
#API request
table_ny_taxi_tripdata = client.get_table(table_ref)
#to dataframe
df_ny_taxi_tripdata = client.list_rows(table=table_ny_taxi_tripdata, max_results=5).to_dataframe()
df_ny_taxi_tripdata.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00290,2019-02-01 00:13:23+00:00,2019-02-01 00:29:01+00:00,73.0,73.0,NaN,B00290
1,B00419,2019-02-01 00:38:11+00:00,2019-02-01 00:59:48+00:00,51.0,235.0,NaN,B00419
2,B00445,2019-02-01 00:47:55+00:00,2019-02-01 00:58:19+00:00,121.0,171.0,NaN,B00445
3,B00628,2019-02-01 00:04:25+00:00,2019-02-01 01:03:32+00:00,13.0,84.0,NaN,B00628
4,B00628,2019-02-01 00:25:52+00:00,2019-02-01 01:06:06+00:00,88.0,239.0,NaN,B00628


In [34]:
# amount of data scanned in MB
def data_scanned(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job_bytes = client.query(query, job_config=dry_run_config, location="asia-southeast1").total_bytes_processed
    print(f'Data processed: {round(query_job_bytes / 10**6, 3)} MB')

def runtime(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    client.query(query, job_config=time_config, location="asia-southeast1").result()
    end = time()
    print(f'Runtime is: {round(end-start, 3)} seconds')

## Question 1:
What is the count for fhv vehicle records for year 2019?
- 65,623,481
- 43,244,696 [`ANSWER`]
- 22,978,333
- 13,942,414

In [15]:
# query
query_q1 = """
                SELECT COUNT(1) as num_count
                FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
        """
        
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results = client.query(query, job_config=safe_config)
#convert to dataframe
df_query_results = query_results.to_dataframe()
df_query_results.head()



,num_count
0,43244696


In [16]:
data_scanned(query=query_q1)
runtime(query=query_q1)

Data processed: 0.0 MB
Runtime is: 1.077 seconds


## Question 2:
Write a query to count the distinct number of `affiliated_base_number` for the entire dataset on both the tables.</br> 
What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

- 25.2 MB for the External Table and 100.87MB for the BQ Table
- 225.82 MB for the External Table and 47.60MB for the BQ Table
- 0 MB for the External Table and 0MB for the BQ Table
- 0 MB for the External Table and 317.94MB for the BQ Table [`ANSWER`] [`External: 0.0 MB, BQ: 333.384 MB`]

In [40]:
# for External Table
# amount of data scanned in MB
def data_scanned_external(query, table_id, external_config):
    dry_run_config = bigquery.QueryJobConfig(table_definitions={table_id: external_config}, dry_run=True)
    query_job_bytes = client.query(query, job_config=dry_run_config, location="asia-southeast1").total_bytes_processed
    print(f'Data processed: {round(query_job_bytes / 10**6, 3)} MB')
# time it takes to complete the query
def runtime_external(query, table_id, external_config):
    time_config = bigquery.QueryJobConfig(table_definitions={table_id: external_config}, use_query_cache=False)
    start = time()
    client.query(query, job_config=time_config, location="asia-southeast1").result()
    end = time()
    print(f'Runtime is: {round(end-start, 3)} seconds')

In [87]:
#  using EXTERNAL TABLE
#  https://cloud.google.com/bigquery/docs/external-data-cloud-storage#query_a_temporary_external_table
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
# Configure the external data source and query job.
external_config = bigquery.ExternalConfig("PARQUET")
external_config.source_uris = [
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-01.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-03.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-04.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-05.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-06.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-07.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-08.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-09.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-10.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-11.parquet",
    "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-12.parquet",
]
external_config.schema = [
    bigquery.SchemaField("dispatching_base_num", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("pickup_datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("dropOff_datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("PUlocationID", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("DOlocationID", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField(
        "SR_Flag",
        "FLOAT",
        mode="NULLABLE",
    ),
    bigquery.SchemaField("Affiliated_base_number", "STRING", mode="NULLABLE"),
]
# external_config.options.skip_leading_rows = 1
table_id = "ny_taxi"
job_config = bigquery.QueryJobConfig(table_definitions={table_id: external_config}, maximum_bytes_billed = 10**9)

# Count the distinct number of records for `Affiliated_base_number`
query_op1_q1 = f"SELECT COUNT(1) FROM `{table_id}`"

query_job = client.query(
    query_op1_q1, job_config=job_config, location="asia-southeast1", 
).to_dataframe()
query_job.head()


,f0_
0,43244696


In [88]:
# check the time it takes to query
runtime_external(query_op1_q1, table_id, external_config)
# check the estimated amount of data
data_scanned_external(query_op1_q1, table_id, external_config)

Runtime is: 1.336 seconds
Data processed: 0.0 MB


In [91]:
# query using Bigquery

query_op2_q2 = """
                SELECT 
                    DISTINCT affiliated_base_number, 
                    COUNT(1) AS num
                FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                GROUP BY 1
  """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results_q2 = client.query(query_op2_q2, job_config=safe_config)
#convert to dataframe
df_query_results_q2 = query_results_q2.to_dataframe()
df_query_results_q2.head()

,Affiliated_base_number,num
0,B00290,8258
1,B00419,19144
2,B00445,87793
3,B00628,62926
4,B00446,30236


In [ ]:
# check the time it takes to query
runtime(query_op2_q2)
# check the estimated amount of data
data_scanned(query_op2_q2)

## Question 3:
How many records have both a blank (null) PUlocationID and DOlocationID in the entire dataset?
- 717,748 [`ANSWER`] [`717748`]
- 1,215,687
- 5
- 20,332

In [94]:
# query using Bigquery

query_q3 = """
                SELECT 
                    PUlocationID,	
                    DOlocationID,	 
                    COUNT(1) AS num
                FROM 
                  `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                WHERE 
                  PUlocationID IS NULL AND DOlocationID IS NULL
              GROUP BY 
                  PUlocationID,	
                  DOlocationID
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results_q3 = client.query(query_q3, job_config=safe_config)
#convert to dataframe
df_query_results_q3 = query_results_q3.to_dataframe()
df_query_results_q3.head()

,PUlocationID,DOlocationID,num
0,NaN,NaN,717748


## Question 4: 
What is the best strategy to optimize the table if query always filter by `pickup_datetime` and order by `affiliated_base_number`?
- baseline [Runtime is: 7.666 seconds | Data processed: 2091.336 MB]
- Cluster on `pickup_datetime` Cluster on `affiliated_base_number` [Runtime is: 0.959 seconds | Data processed: 2091.336 MB]
- Partition by `pickup_datetime` Cluster on `affiliated_base_number` [Runtime is: 0.945 seconds | Data processed: 1117.192 MB]
- Partition by `pickup_datetime` Partition by `affiliated_base_number` [https://cloud.google.com/bigquery/docs/partitioned-tables#limitations]
- Partition by `affiliated_base_number` Cluster on `pickup_datetime`
 
        [https://cloud.google.com/bigquery/docs/partitioned-tables#limitations] 
        [`It might be possible if we convert the affiliated_base_number to INTEGER from STRING`]

resource: https://towardsdatascience.com/how-to-use-partitions-and-clusters-in-bigquery-using-sql-ccf84c89dd65

In [112]:
# BASELINE - Query without PARTITION AND CLUSTER
# Query 
query_baseline = """
                  SELECT *
                  FROM 
                    `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                  WHERE 1=1 
                    AND DATE(pickup_datetime) BETWEEN "2019-01-01" AND "2019-01-31"
                    AND affiliated_base_number = "B00445"
              
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
results_query_baseline = client.query(query_baseline, job_config=safe_config)
#convert to dataframe
df_results_query_baseline = results_query_baseline.to_dataframe()
df_results_query_baseline.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00445,2019-01-01 00:32:02+00:00,2019-01-01 00:56:51+00:00,145.0,16.0,NaN,B00445
1,B00445,2019-01-01 00:25:50+00:00,2019-01-01 00:33:58+00:00,171.0,15.0,NaN,B00445
2,B00445,2019-01-01 00:45:47+00:00,2019-01-01 01:03:04+00:00,252.0,82.0,NaN,B00445
3,B00445,2019-01-01 00:36:25+00:00,2019-01-01 01:00:17+00:00,265.0,92.0,NaN,B00445
4,B00445,2019-01-01 00:53:37+00:00,2019-01-01 01:37:09+00:00,229.0,15.0,NaN,B00445


In [113]:
# check the time it takes to query
runtime(query_baseline)
# check the estimated amount of data
data_scanned(query_baseline)

Runtime is: 7.666 seconds
Data processed: 2091.336 MB


In [111]:
# OPTION 1 - Cluster on `pickup_datetime` Cluster on `affiliated_base_number` 
# create a table cluster 
cluster_op1_q4 = """
                CREATE OR REPLACE TABLE 
                    `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_clustered_2`
                CLUSTER BY
                    pickup_datetime, affiliated_base_number AS (
                        SELECT *
                        FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                    )
          """
# Limit results to 1GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
# Query
results_cluster_op1_q4 = client.query(cluster_op1_q4, job_config=safe_config).result()


# Check this link if the clustered table was added
# https://console.cloud.google.com/bigquery?project=dtc-de-2023&ws=!1m5!1m4!4m3!1sdtc-de-2023!2sny_taxi!3sny_taxi_tripdata_2019_clustered&authuser=3

In [114]:
# OPTION 1 - Cluster on `pickup_datetime` Cluster on `affiliated_base_number` 
# Query clustered table to check
query_op1_q4 = """
                  SELECT *
                  FROM 
                    `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_clustered_2`
                  WHERE 1=1 
                    AND DATE(pickup_datetime) BETWEEN "2019-01-01" AND "2019-01-31"
                    AND affiliated_base_number = "B00445"
              
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
results_query_op1_q4 = client.query(query_op1_q4, job_config=safe_config)
#convert to dataframe
df_results_query_op1_q4 = results_query_op1_q4.to_dataframe()
df_results_query_op1_q4.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00445,2019-01-01 02:34:30+00:00,2019-01-01 03:09:58+00:00,7.0,252.0,NaN,B00445
1,B00445,2019-01-01 10:50:03+00:00,2019-01-01 11:07:01+00:00,7.0,252.0,NaN,B00445
2,B00445,2019-01-01 05:37:01+00:00,2019-01-01 05:46:56+00:00,9.0,252.0,NaN,B00445
3,B00445,2019-01-01 07:41:59+00:00,2019-01-01 08:23:01+00:00,9.0,92.0,NaN,B00445
4,B00445,2019-01-01 03:00:27+00:00,2019-01-01 03:14:30+00:00,9.0,7.0,NaN,B00445


In [115]:
# check the time it takes to query
runtime(query_op1_q4)
# check the estimated amount of data
data_scanned(query_op1_q4)

Runtime is: 0.959 seconds
Data processed: 2091.336 MB


In [118]:
# OPTION 2 - Partition by `pickup_datetime` Cluster on `affiliated_base_number`
# create a partition and cluster table 
part_clus_op2_q4 = """
                    CREATE OR REPLACE TABLE 
                        `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op2`
                    PARTITION BY
                        DATE(pickup_datetime)
                    CLUSTER BY
                        affiliated_base_number AS (
                            SELECT *
                            FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                        )
          """
# Limit results to 1GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
# Query
results_part_clus_op2_q4 = client.query(part_clus_op2_q4, job_config=safe_config).result()


# Check this link if the clustered table was added
# https://console.cloud.google.com/bigquery?project=dtc-de-2023&ws=!1m5!1m4!4m3!1sdtc-de-2023!2sny_taxi!3sny_taxi_tripdata_2019_clustered&authuser=3

In [119]:
# OPTION 2 - Partition by `pickup_datetime` Cluster on `affiliated_base_number`
# Query partition then clustered table to check
query_op2_q4 = """
                  SELECT *
                  FROM 
                    `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op2`
                  WHERE 1=1 
                    AND DATE(pickup_datetime) BETWEEN "2019-01-01" AND "2019-01-31"
                    AND affiliated_base_number = "B00445"
              
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
results_query_op2_q4 = client.query(query_op2_q4, job_config=safe_config)
#convert to dataframe
df_results_query_op2_q4 = results_query_op2_q4.to_dataframe()
df_results_query_op2_q4.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00445,2019-01-20 11:57:54+00:00,2019-01-20 12:25:39+00:00,7.0,64.0,NaN,B00445
1,B00445,2019-01-20 04:31:52+00:00,2019-01-20 04:52:41+00:00,7.0,15.0,NaN,B00445
2,B00445,2019-01-20 22:09:19+00:00,2019-01-20 22:17:19+00:00,9.0,15.0,NaN,B00445
3,B00445,2019-01-20 17:24:51+00:00,2019-01-20 17:38:05+00:00,9.0,265.0,NaN,B00445
4,B00445,2019-01-20 00:33:00+00:00,2019-01-20 00:38:39+00:00,9.0,16.0,NaN,B00445


In [121]:
# check the time it takes to query
runtime(query_op2_q4)
# check the estimated amount of data
data_scanned(query_op2_q4)

Runtime is: 0.945 seconds
Data processed: 1117.192 MB


In [154]:
# OPTION 3 - Partition by `pickup_datetime` Partition by `affiliated_base_number`
# create a partition table on two columns 
part_op3_q4_1 = """
                    CREATE OR REPLACE TABLE 
                        `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op3` 
                    PARTITION BY
                        DATE(pickup_datetime)  AS 
                        (
                            SELECT *
                            FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                        )
          """
# Limit results to 1GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
# Query
results_part_op3_q4_1 = client.query(part_op3_q4_1, job_config=safe_config).result()

# Check this link if the clustered table was added
# https://console.cloud.google.com/bigquery?project=dtc-de-2023&ws=!1m5!1m4!4m3!1sdtc-de-2023!2sny_taxi!3sny_taxi_tripdata_2019_clustered&authuser=3

In [ ]:
# OPTION 3 - Partition by `pickup_datetime` Partition by `affiliated_base_number` [NOT POSSIBLE TO PARTITION ON TWO COLS SEE NOTES]
# create a partition table on two columns 
part_op3_q4_2 = """
                    CREATE OR REPLACE TABLE 
                        `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op3_1`
                    PARTITION BY
                        affiliated_base_number OPTIONS(partition_expiration_days=90) AS 
                        (
                            SELECT *
                            FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op3`
                        )
          """
# Limit results to 1GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
# Query
results_part_op3_q4_2 = client.query(part_op3_q4_2, job_config=safe_config).result()


# Check this link if the clustered table was added
# https://console.cloud.google.com/bigquery?project=dtc-de-2023&ws=!1m5!1m4!4m3!1sdtc-de-2023!2sny_taxi!3sny_taxi_tripdata_2019_clustered&authuser=3

In [ ]:
# OPTION 4 - Partition by `affiliated_base_number` Cluster on `pickup_datetime` [NOT POSSIBLE]
# create a partition and cluster table 
part_clus_op4_q4 = """
                    CREATE OR REPLACE TABLE 
                        `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op4`
                    PARTITION BY
                        affiliated_base_number
                    CLUSTER BY
                        DATE(pickup_datetime) AS (
                            SELECT *
                            FROM `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                        )
          """
# Limit results to 1GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
# Query
results_part_clus_op4_q4 = client.query(part_clus_op4_q4, job_config=safe_config).result()


# Check this link if the clustered table was added
# https://console.cloud.google.com/bigquery?project=dtc-de-2023&ws=!1m5!1m4!4m3!1sdtc-de-2023!2sny_taxi!3sny_taxi_tripdata_2019_clustered&authuser=3

## Question 5:
Implement the optimized solution you chose for **question** 4. Write a query to retrieve the distinct `affiliated_base_number` between `pickup_datetime` 2019/03/01 and 2019/03/31 (inclusive).</br> 
Use the BQ table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 4 and note the estimated bytes processed. What are these values? Choose the answer which most closely matches.
- 12.82 MB for non-partitioned table and 647.87 MB for the partitioned table
- 647.87 MB for non-partitioned table and 23.06 MB for the partitioned table [`ANSWER`]  [`679.342 MB | 24.173 MB`]

- 582.63 MB for non-partitioned table and 0 MB for the partitioned table
- 646.25 MB for non-partitioned table and 646.25 MB for the partitioned table

DETAILS:
NON-PARTITIONED - [Runtime is: 1.419 seconds | Data processed: 679.342 MB]
PARTITIONED - [Runtime is: 1.007 seconds | Data processed: 24.173 MB]

In [133]:
# BASELINE - Query without PARTITION AND CLUSTER
# Query 
query_baseline_q5 = """
                        SELECT 
                            DISTINCT affiliated_base_number, 
                            DATE(pickup_datetime) AS pickup_datetime,
                            COUNT(1) AS num
                        FROM 
                            `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019`
                        WHERE 1=1 
                            AND DATE(pickup_datetime) BETWEEN "2019-03-01" AND "2019-03-31"
                        GROUP BY
                            affiliated_base_number, 
                            pickup_datetime
              
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**12)
#query
results_query_baseline_q5 = client.query(query_baseline_q5, job_config=safe_config)
#convert to dataframe
df_results_query_baseline_q5 = results_query_baseline_q5.to_dataframe()
df_results_query_baseline_q5.head()

,affiliated_base_number,pickup_datetime,num
0,B00477,2019-03-01,215
1,B01087,2019-03-01,815
2,B01215,2019-03-01,262
3,B01233,2019-03-01,178
4,B01364,2019-03-01,75


In [134]:
# check the time it takes to query
runtime(query_baseline_q5)
# check the estimated amount of data
data_scanned(query_baseline_q5)

Runtime is: 1.419 seconds
Data processed: 679.342 MB


In [135]:
# USING Q4-OPTION2 - Query with PARTITION AND CLUSTER
# Query 
query_part_q5 = """
                        SELECT 
                            DISTINCT affiliated_base_number, 
                            DATE(pickup_datetime) AS pickup_datetime,
                            COUNT(1) AS num
                        FROM 
                            `dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019_op2`
                        WHERE 1=1 
                            AND DATE(pickup_datetime) BETWEEN "2019-03-01" AND "2019-03-31"
                        GROUP BY
                            affiliated_base_number, 
                            pickup_datetime
              
          """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**12)
#query
results_query_part_q5 = client.query(query_part_q5, job_config=safe_config)
#convert to dataframe
df_results_query_part_q5 = results_query_part_q5.to_dataframe()
df_results_query_part_q5.head()

,affiliated_base_number,pickup_datetime,num
0,B02617,2019-03-27,114
1,B02867,2019-03-27,193
2,B02869,2019-03-27,252
3,B02872,2019-03-27,405
4,B02790,2019-03-27,5


In [137]:
# check the time it takes to query
runtime(query_part_q5)
# check the estimated amount of data
data_scanned(query_part_q5)

Runtime is: 1.007 seconds
Data processed: 24.173 MB


## Question 6: 
Where is the data stored in the External Table you created?

- Big Query 
- GCP Bucket [`ANSWER`] [https://cloud.google.com/bigquery/docs/external-tables]
- Container Registry
- Big Table

## Question 7:
It is best practice in Big Query to always cluster your data:
- True [`ANSWER`] - TO MINIMIZE COST AND MAXIMIZE EFFICIENCY
- False

## (Not required) Question 8:
A better format to store these files may be parquet. Create a data pipeline to download the gzip files and convert them into parquet. Upload the files to your GCP Bucket and create an External and BQ Table. 

ANSWER:

I did the approach using the csv.gz and caused me a lot of headache dealing with data types. It prompts if me error on
the timestamp of `pickup_datetime` and `dropoff_datetime`. 

What works for me: (DRAFT NOTES)
1. Save the file in csv.gz
2. tweak the csv.gz to hard encode the data type except the date, for the date we pass it to parse_dates
3. on parse_dates, the column is based on index 0. 
4. I save this to parquet files to save some bytes
5. Hard coded the schema field before transfer to bigquery
-after a lot of coffee and walking I have finally fetch the files from github, upload this files to GCS, then transfer this to BIGQUERY. whooosh


In [ ]:
#create a client
client = bigquery.Client()
# Construct a reference to the "ny_taxi_bucket_de_2023" dataset
dataset_ref = client.dataset(dataset_id="dlb_lake", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference
table_ref = dataset_ref.table("staging_user_sessions")
#API request
table_staging_user_sessions = client.get_table(table_ref)
#to dataframe
df_table_staging_user_sessions = client.list_rows(table=table_staging_user_sessions, max_results=5).to_dataframe()
df_table_staging_user_sessions.head()

In [7]:
# USING Q4-OPTION2 - Query with PARTITION AND CLUSTER
# Query 
query_dedup = """ 
            -- CREATE A CTE TABLE
            CREATE OR REPLACE TABLE `dtc-de-2023.dlb_lake.user_sessions` AS 
                WITH CTE1 AS (
                SELECT 
                    *, 
                    ROW_NUMBER() OVER(
                                        PARTITION BY user_session_id 
                                        ORDER BY _bq_inserted_at) AS latest_row
                    FROM `dtc-de-2023.dlb_lake.staging_user_sessions`
                )

            -- FETCH ONLY THE LATEST ROW WHICH IS THE LATEST BQ INSERTED TIMESTAMP
            SELECT * EXCEPT (latest_row)
            FROM CTE1
            WHERE latest_row = 1 
            """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**12)
#query
results_query_dedup = client.query(query_dedup, job_config=safe_config)
